In [ ]:
from datetime import datetime
import os

from bokeh.models import DatetimeTickFormatter, PrintfTickFormatter
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd
import tables as tb

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200128")
date_str = "20200129_013952"

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")

# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]
#print(keys)

In [ ]:
x_data = []
y_data = []
frequencies = None
datetimes = []
#print(keys)
for i, key in enumerate(keys):
    pd_h5 = pd.read_hdf(ifname_psd, key=key)
    x_data.append(pd_h5.X.values)
    y_data.append(pd_h5.Y.values)
    datetimes.append(datetime.strptime(key.replace('DT_', ''), '%Y%m%d_%H%M%S'))
    if i == 0:
        frequencies = pd_h5.Frequencies.values

In [ ]:
x = np.stack(x_data)
y = np.stack(y_data)
ts = np.array(datetimes, dtype=np.datetime64)

In [ ]:
psd_x = hv.Image((frequencies, ts, x), rtol=1, label='PSD X').opts(width=460)
psd_y = hv.Image((frequencies, ts, y), rtol=1, label='PSD Y').opts(yaxis='bare', width=350)

In [ ]:
psd = psd_x + psd_y

colorbar_opts = {'colorbar': True, 'colorbar_position': 'bottom',
                 'colorbar_opts': {'formatter': PrintfTickFormatter(format="%0.0e")}}

dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
date_formatter = DatetimeTickFormatter(**formats)

psd.opts(opts.Image(cmap='viridis', logz=True, invert_yaxis=True,
                    **colorbar_opts,
                    height=400,
                    yformatter=date_formatter,
                    xlabel='𝜈 (Hz)', ylabel=''))